In [2]:
# テキストコーパスをチャンクに分割
with open('kitei.txt', 'r', encoding='utf-8') as f:
    text = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(text)

In [3]:
print(type(texts))
print("--------")
print(len(texts))
print("--------")
print(texts[0])
print("--------")
print(texts[1])

<class 'list'>
--------
19
--------
株式会社ミライソフト 社内規定

■第1章 総則
●第1条（目的）
この規定は、株式会社ミライソフト（以下「当社」という）の円滑な業務運営と、全従業員の安全かつ公正な職場環境の確保を目的とする。
--------
●第2条（適用範囲）
本規定は、当社に雇用される全ての従業員に適用する。


In [4]:
# パッセージのベクトル化
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large', model_kwargs={'device': 'cpu'})

In [5]:
from langchain_community.vectorstores import FAISS

# データベースの保存
db = FAISS.from_texts(texts, embeddings)
db.save_local('kitei.db')

In [6]:
# 保存したデータベースの読み込み
db = FAISS.load_local('kitei.db',embeddings, allow_dangerous_deserialization=True)

In [7]:
similarity_sample = db.similarity_search("勤務")
print(len(similarity_sample))
print("--------")
print(type(similarity_sample[0]))
print("--------")
print(similarity_sample[0].page_content)

4
--------
<class 'langchain_core.documents.base.Document'>
--------
■第2章 勤務
●第3条（勤務時間）
始業時刻：午前9時00分
終業時刻：午後6時00分
休憩時間：正午12時から午後1時まで（60分）


In [8]:
# 検索器の構築
retriever = db.as_retriever()   # 検索文書数 4（デフォルト）

In [9]:
# モデルの準備
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

#CAのモデルを使用
model_id = "cyberagent/open-calm-small"
tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    ).eval()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.01,
    repetition_penalty=2.0,
    )

Device set to use cpu


In [10]:
# プロンプトの準備

template = """
ユーザー:以下のテキストを参照して、それに続く質問に答えてください。

{context}

{question}

システム:"""

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
    )

In [11]:
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline

qa = RetrievalQA.from_chain_type(
    llm=HuggingFacePipeline(pipeline=pipe),
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
    )

In [12]:
# 実行例

q = "勤務時間は何時から何時までですか？"
ans = qa.invoke(q)
print(ans['result'])

print("--------------------------")

import re
pattern = re.compile(r'システム:(.*)',re.DOTALL)
match = pattern.search(ans['result'])
ans0 = match.group(1)
print(ans0)



> Entering new RetrievalQA chain...

> Finished chain.

ユーザー:以下のテキストを参照して、それに続く質問に答えてください。

■第2章 勤務
●第3条（勤務時間）
始業時刻：午前9時00分
終業時刻：午後6時00分
休憩時間：正午12時から午後1時まで（60分）

●第7条（リモートワーク）
原則として週2回までのリモートワークを認める。必要機材は会社より貸与する。

上限を超える場合、事前に上長の承認を得ること
日当：出張日1日につき2,000円を支給（食費等の雑費含む）

業務上の背任行為や横領、重大な過失

勤務時間は何時から何時までですか？

システム:リモートでの勤務です。(PC・タブレット等の端末からの操作となります。)
--------------------------
リモートでの勤務です。(PC・タブレット等の端末からの操作となります。)
